# あなたの文章に合った「いらすとや」画像をレコメンド♪（Sentence-BERT編）

解説記事: https://qiita.com/sonoisa/items/1df94d0a98cd4f209051

## アルゴリズムの概要

本アプリの基本的なアイディアは次のとおりです。

1. 与えられた文や画像の説明文を、それぞれSentence-BERTを用いて文の分散表現（つまりはベクトル）に変換する。
1. 与えられた文と画像の説明文の意味の近さを、それぞれの文の分散表現を使って計算する（意味の近さ = 2つのベクトルのなす角の小ささ = コサイン類似度の大きさとする）。
1. コサイン類似度が大きい説明文を持つ画像トップN個を選ぶことで、与えられた文と意味が近い画像を発見できる。

模式図にすると、次のようになります。

<img src="https://camo.qiitausercontent.com/fb62a6b8a0fd447e1ff1370e83ff0b636a3f9a36/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f32363036322f33393935316266392d663630322d383565332d646239662d6235353764663164376266642e706e67" width="800">

## 準備

### パスワード設定

**※勉強会に用いるデータセットの解凍にはパスワードが必要です（一般には公開しません）。**  
**※次の変数 RESOURCES_PASSWORD に解凍用の秘密のパスワードを設定してから、以降の処理を実行してください。**

In [2]:
# データセットの解凍用パスワード
RESOURCES_PASSWORD = ""

### 依存ライブラリのインストール

インストールに5分程度かかります。気長にお待ちください。

In [4]:
!pip install -q transformers==4.7.0 fugashi ipadic gdown

### 「いらすとや」さんの画像メタデータのダウンロード

In [5]:
!gdown "https://drive.google.com/uc?export=view&id=1NQ66ZynRY63SIlk2i4OhMj837YDucLR9"

Downloading...
From: https://drive.google.com/uc?export=view&id=1NQ66ZynRY63SIlk2i4OhMj837YDucLR9
To: /content/ii20210224.zip
100% 3.63M/3.63M [00:00<00:00, 181MB/s]


### データの解凍

In [6]:
!unzip -P {RESOURCES_PASSWORD} ii20210224.zip

Archive:  ii20210224.zip
  inflating: irasuto_items.json      


### 画像メタデータを読み込む

LINEスタンプはイラストではないため除外します。

In [8]:
import json

with open('irasuto_items.json', 'r', encoding="utf-8") as items_file:
    items = json.load(items_file)

items = [item for item in items \
             if "LINEスタンプ" not in item["title"] and \
             "LINEのスタンプ" not in item["title"]]

### 正規化処理の定義

neologdの正規化処理を少し変えたものを利用します。

- neologdの正規化処理: https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja

In [11]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    s = s.upper()
    return s

def normalize_text(text):
    return normalize_neologd(text)

### タイトルや説明文の「〜のイラスト」などの冗長な表現を削除する前処理

In [12]:
def normalize_title(title):
    title = title.strip()
    
    match = re.match(r"^「([^」]+)」$", title)
    if match:
        title = match.group(1)

    match = re.match(r"^POP素材「([^」]+)」$", title)
    if match:
        title = match.group(1)
    
    title = re.sub(r"(の?(?:イラスト|イラストの|イラストト|イ子のラスト|イラス|イラスト文字|「イラスト文字」|イラストPOP文字|ペンキ文字|タイトル文字|イラスト・メッセージ|イラスト文字・バナー|キャラクター(たち)?|マーク|アイコン|シルエット|シルエット素材|フレーム（枠）|フレーム|フレーム素材|テンプレート|パターン|パターン素材|ライン素材|コーナー素材|リボン型バナー|評価スタンプ|背景素材))+(\s*([0-9０-９]*|その[0-9０-９]+))(です。)?", "", title)
    
    title = normalize_text(title)
    
    if title.strip() == "":
        raise ValueError(title)
    
    return title

### タイトルと説明文の正規化を実行

説明文がなければタイトルを説明文の代わりにします。

In [14]:
for item in items:
    try:
        title = item["title"]
        normalized_title = normalize_title(title)
        item["normalized_title"] = normalized_title

        desc = item["desc"]
        if desc.strip() == "":
            # 説明文がない場合は、タイトルを説明文にする
            item["normalized_desc"] = normalized_title
            item["desc"] = title
        else:
            normalized_desc = normalize_title(desc)
            item["normalized_desc"] = normalized_desc
            # print(desc, normalized_desc)
    except:
        continue


### Sentence-BERTクラスの定義

In [15]:
from transformers import BertJapaneseTokenizer, BertModel
import torch


class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)

Sentence-BERTモデルを読み込む

In [16]:
model = SentenceBertJapanese("sonoisa/sentence-bert-base-ja-mean-tokens")

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

### 文の分散表現の計算方法の定義

与えられた文を、文の分散表現に変換する関数 get_sentence_vector を定義します。  
今回採用した文の分散表現の計算方法は次の通りです。

1. 正規化を行う。
2. 1の文をSentence-BERTを用いて文の分散表現を計算する。

In [26]:
def get_sentence_vector(sentence):
    sentence = normalize_text(sentence)
    return model.encode([sentence])[0].numpy()

試しに文ベクトルを計算してみます。

In [27]:
get_sentence_vector("与えられた文から文の分散表現を計算します。")

array([-9.07196164e-01,  5.25513172e-01, -1.59523058e+00, -1.04668438e+00,
       -7.37658501e-01,  4.52362657e-01, -3.74935150e-01,  2.58568197e-01,
        2.98401900e-03,  6.77360773e-01,  9.50184107e-01,  1.24125063e-01,
       -6.43323600e-01,  5.91418371e-02, -1.96929872e-01, -1.85807526e-01,
        8.88445042e-03,  4.27624047e-01,  5.24865910e-02,  9.10040498e-01,
        1.22919559e+00,  2.63807476e-01,  1.04527056e+00,  3.72870490e-02,
        4.92842019e-01, -1.20458901e-01, -1.69820458e-01, -1.04896438e+00,
       -1.72844231e-02, -2.08393514e-01, -5.23855150e-01, -2.50866413e-02,
       -9.69365537e-01, -5.82301378e-01,  2.11963534e-01,  6.73911929e-01,
        6.10183358e-01, -6.55089200e-01, -2.06699863e-01,  4.44268167e-01,
       -1.11041689e+00, -6.15052938e-01, -4.94100690e-01, -4.03977185e-02,
        6.49147034e-02,  2.19064808e+00, -4.47669998e-02,  4.31178212e-01,
       -2.18916893e-01,  4.08492863e-01,  8.20284605e-01, -1.19899631e-01,
       -6.76331997e-01,  

## 説明文の分散表現の計算実行

画像メタデータに説明文の分散表現を追加します。

In [28]:
from tqdm import tqdm
for item in tqdm(items):
    desc = item["desc"]
    desc_vec = get_sentence_vector(desc)
    item["vec"] = desc_vec

100%|██████████| 25007/25007 [04:44<00:00, 87.99it/s]


## コサイン類似度の定義

今回は、文の意味の近さを、文の分散表現のコサイン類似度によって測ります。  
文の意味が近ければ、文の分散表現（ベクトル）v1とv2が近くなるという定性的性質を、ベクトルの成す角のcosによって測るということです。

In [29]:
import numpy as np

def cos_sim(v1, v2):
    v1 = v1 / np.linalg.norm(v1, axis=0, ord=2)
    v2 = v2 / np.linalg.norm(v2, axis=0, ord=2)
    return np.sum(v1 * v2)

## 画像検索結果GUIの定義

最後のステップです。画像を検索する関数を定義します。  
いままで作った関数を使えば、次の処理からなる検索アルゴリズム（最初の図も参照）を簡単に実装できますね。  

1. 与えられた文から文の分散表現を計算する。
2. その分散表現と、説明文の分散表現の間のコサイン類似度を計算する。
3. コサイン類似度の高い順に画像の関連情報を表示する。

**※なお「いらすとや」さんの広告収入モデルに悪影響を与えないよう、必ず「いらすとや」さんのページへのリンクを張り、画像のダウンロードは「いらすとや」さんのページから行うようにしましょう。その他、[「いらすとや」さんの利用規約](https://www.irasutoya.com/p/terms.html)に違反しないよう十分ご注意ください。**


In [30]:
from IPython.display import display, HTML, clear_output
from html import escape
import numpy as np

def search_irasuto(sentence, top_n=3):
    sentence_vector = get_sentence_vector(sentence)
    sims = []
    if sentence_vector is None:
        print("検索できない文章です。もう少し文章を長くしてみてください。")
    else:
        for item in items:
            v = item["vec"]
            if v is None:
                sims.append(-1.0)
            else:
                sim = cos_sim(sentence_vector, v)
                sims.append(sim)
    
    count = 0
    for index in np.argsort(sims)[::-1]:
        if count >= top_n:
            break
        item = items[index]
        desc = escape(item["desc"])
        imgs = item["imgs"]
        if len(imgs) == 0:
            continue
        img = imgs[0]
        page = item["page"]
        sim = sims[index]
        display(HTML("<div><a href='" + page + "' target='_blank' rel='noopener noreferrer'><img src='" + img + "' width='100'>" + str(sim) + ": " + desc + "</a><div>"))
        count += 1

## アプリの動作確認

さあ、これでアルゴリズムは完成しました。早速、試してみましょう。  

In [31]:
search_irasuto(sentence="暴走したAI", top_n=5)

In [32]:
search_irasuto(sentence="リモートワークで勉強会", top_n=5)

In [34]:
search_irasuto(sentence="いらすとやさんに惜しみない拍手を", top_n=5)